<a href="https://colab.research.google.com/github/lavstat17/Notebook_Product_scoring/blob/main/scoring_organic_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing libraries
import pandas as pd
import math
import warnings
import re,logging
warnings.filterwarnings("ignore")

class product_score:
  def __init__(self):
    self.log = logging.getLogger(__name__)

  # def decrease_score(self,row):
  #   if row['category'].str.contains("organic"):
  #     return row['final_score'] *90/100
  #   elif row['category'].str.contains("made with organic"):
  #     return row['final_score']*75/100
  #   elif row['category'].str.contains("ingredients panel"):
  #     return row['final_score']*50/100

  def integration(self):
    #Read the Scrapped data Vitaminshoppe and chetoos
    df = pd.read_csv(r'/content/drive/MyDrive/organic_snacks_data_analysis.csv')
    #convert each serving size to 100g
    df_new = df[['protein','Saturatedfat','sugars','Servingsize','sodium','fiber','calories']].astype(int)
    d = {4:25, 8:12.5, 12:8.4, 12.4:8.1, 13:7.7, 14:7.2, 15:6.67, 16:6.3, 17:5.9, 18:5.6, 19:5.31, 19.8:5.1, 20:5, 21:4.8, 22:4.55, 23:4.35, 24:4.2, 25:4, 26:3.85, 27:3.71, 28:3.58, 29:3.45, 30:3.34, 31:3.23, 32:3.13, 33:3.04, 34:2.95, 35:2.87, 36:2.8, 37:2.71, 38:2.65, 39:2.57, 40:2.5, 41:2.45, 42:2.39, 43:2.33, 44:2.28, 45:2.23}
    df_new['constant'] = df['Servingsize'].map(d)
    df_new['protein_100g'] = df_new['protein']*df_new['constant'].astype(int)
    df_new['Saturatedfat_100g'] = df_new['Saturatedfat']*df_new['constant'].astype(int)
    df_new['sugars_100g'] = df_new['sugars']*df_new['constant'].round(decimals=2)
    df_new['fiber_100g'] = df_new['fiber']*df_new['constant'].round(decimals=2)
    df_new['Servingsize_100g'] = df_new['Servingsize']*df_new['constant'].round(decimals=2)
    df_new['sodium_mg'] = df_new['sodium'].astype(int)
    df_new['Energy_Kcal'] = df_new['calories']*df_new['constant'].round(decimals=2)
    df_new['Energy_KJ'] = df_new['calories']*4.184
    df_new.to_csv('conversion_data.csv')

    #negative factors dataframe
    df_new1= df_new[['Energy_KJ','sodium_mg','Saturatedfat_100g','sugars_100g']]

    ##positive factors dataframe
    df_new2= df_new[['fiber_100g','protein_100g']]

    ## Negative factors scoring based on threshold value
    bins1 = [0,335,670,1005,1340,1675,2010,2345,2680,3015,3350]
    bins2 = [0,1,2,3,4,5,6,7,8,9,10,500]
    bins3 = [0,4.5,9,13.5,18,22.5,27,31,36,40,45,500]
    bins4 = [0,90,180,270,360,450,540,630,720,810,900,1500]
    df_new1['Energy_KJ_score'] = pd.cut(df_new1['Energy_KJ'], bins=bins1, labels=False,include_lowest=True)
    df_new1['Saturated_Fat_score_'] = pd.cut(df_new1['Saturatedfat_100g'],bins=bins2, labels=False,include_lowest=True)
    df_new1['sugars_100g_score'] = pd.cut(df_new1['sugars_100g'],bins=bins3,labels=False,include_lowest=True)
    df_new1['sodium_mg_score'] = pd.cut(df_new1['sodium_mg'],bins=bins4,labels=False,include_lowest=True)

    ## Positive factors scoring based on threshold values
    bin1 = [0,0.7,1.4,2.1,2.8,3.5,500]
    bin2 = [0,1.6,3.2,4.8,6.4,8.0,500]
    df_new2['fiber_score'] = pd.cut(df_new2['fiber_100g'], bins=bin1, labels=False,include_lowest=True)
    df_new2['protein_score'] = pd.cut(df_new2['protein_100g'],bins=bin2, labels=False,include_lowest=True)
    return df,df_new1,df_new2

  #calculating A points
  #Total 'A' points = (points for energy) + (points for saturated fat) + (points for sugars) + (points for sodium)
  def A_points(self,row):
    return row['Energy_KJ_score']+row['sodium_mg_score']+row['Saturated_Fat_score_']+row['sugars_100g_score']

  #caluclating C points
  #Total 'C' points = (points for % fruit, vegetable & nut content) + (points for fibre [either NSP or AOAC]) + (points for protein)
  def C_points(self,row):
    return row['fiber_score']+row['protein_score']

  # Based on conditions finding overall score
  def overall_score(self,row):
    if row['A'] < 11 or row['A'] >= 11:
      return row['A']- row['C']
  
  def perform_score(self,schedule_app=None):
    df,df_new1,df_new2 = self.integration()
    A = self.A_points(df_new1)
    print(A)
    C = self.C_points(df_new2)
    print(C)

    score = {'A':A,'C':C}
    score = pd.concat(score, axis=1)
    score['final_score'] = score.apply( lambda row : self.overall_score(row), axis = 1)
    print("before reducing the value {}".format(score))

    #Decrease the final_score to 5% for organic products
    score['final_score'] = score['final_score']*95/100
    print("after reducing the score {}".format(score))

    bin2 = [-15,-2,3,11,16,50]
    labels = ['Green','Yellow','Orange','pink','Red']
    
    #Creating 5CNL format
    score['Tag'] = pd.cut(score['final_score'], bins=bin2, labels=labels)
    score = pd.concat([df[['Nameofproduct','category']],score],axis=1)
    score_output = pd.concat([score,df[['calories','sodium','Saturatedfat','sugars','fiber','protein']]],axis=1)

    score_output.to_csv('organic_snacks_scoring.csv')
    print(score_output)


if __name__ == '__main__':
  reco = product_score()
  reco.perform_score()

0      12
1      17
2      15
3      13
4      16
       ..
201    10
202    10
203    12
204    12
205    15
Length: 206, dtype: int64
0      10
1       7
2       1
3       7
4       7
       ..
201     5
202     5
203    10
204     5
205     3
Length: 206, dtype: int64
before reducing the value       A   C  final_score
0    12  10            2
1    17   7           10
2    15   1           14
3    13   7            6
4    16   7            9
..   ..  ..          ...
201  10   5            5
202  10   5            5
203  12  10            2
204  12   5            7
205  15   3           12

[206 rows x 3 columns]
after reducing the score       A   C  final_score
0    12  10         1.90
1    17   7         9.50
2    15   1        13.30
3    13   7         5.70
4    16   7         8.55
..   ..  ..          ...
201  10   5         4.75
202  10   5         4.75
203  12  10         1.90
204  12   5         6.65
205  15   3        11.40

[206 rows x 3 columns]
                             